In [9]:
import glob
import numpy as np
import time
from combine_annotations_into_tiles import combine_annotations_into_tiles
import os
import pickle

In [10]:
# Pre - inputs
pth = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model'
pthim_ann = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\5x'
classcheck = 0


from load_annotation_data import load_annotation_data
# _____________________________________________________________________________

# Inputs
pthDL = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024'
ctlist0, numann0 = load_annotation_data(pthDL, pth, pthim_ann, classcheck)


 
Importing annotation data...
Image 1 of 2: SG_013_0061
 annotation data previously loaded
Image 2 of 2: SG_014_0016
 annotation data previously loaded


In [11]:
#Function
# ______________________________________________________________________________

In [12]:
# Load data from net.pkl file
with open(os.path.join(pthDL, 'net.pkl'), 'rb') as f:
    data = pickle.load(f)
    sxy, nblack, classNames, ntrain, nvalidate = data['sxy'], data['nblack'], data['classNames'], data['ntrain'], data['nvalidate']


In [13]:
if classNames[-1] == "black":
    classNames = classNames[:-1]
print('')

In [14]:
print('Calculating total number of pixels in the training dataset...')
count_annotations = sum(numann0)
annotation_composition = count_annotations / max(count_annotations) * 100
for b, count in enumerate(annotation_composition):
    if annotation_composition[b] == 100:
        print(f' There are {count} pixels of {classNames[b]}. This is the most common class.')
    else:
        print(f' There are {count} pixels of {classNames[b]}, {int(annotation_composition[b])}% of the most common class.')


Calculating total number of pixels in the training dataset...
 There are 19.905351465001388 pixels of islet, 19% of the most common class.
 There are 8.1496239311231 pixels of epithelium, 8% of the most common class.
 There are 13.580334757804946 pixels of vasculature, 13% of the most common class.
 There are 38.320937221401394 pixels of fat, 38% of the most common class.
 There are 17.82459048185089 pixels of acini, 17% of the most common class.
 There are 100.0 pixels of stroma. This is the most common class.
 There are 87.00521934212013 pixels of nontissue, 87% of the most common class.
 There are 35.270251982192015 pixels of PanIN, 35% of the most common class.
 There are 10.700279174113403 pixels of nerves, 10% of the most common class.
 There are 37.76189140810624 pixels of immune, 37% of the most common class.
 There are 9.900705538976249 pixels of PDAC, 9% of the most common class.


In [15]:
# Check for missing annotations
if 0 in count_annotations:
    raise ValueError('There are no annotations for one or more classes. Please add annotations, check nesting, or remove empty classes.')


In [16]:
# Build training tiles
print('')
print('Building training tiles...')
numann0 = np.array(numann0)  # Convert numann0 to a NumPy array
numann = numann0.copy()
percann = np.double(numann0 > 0)
percann = np.dstack((percann, percann))
percann0 = percann.copy()
ty = 'training\\'
obg = os.path.join(pthDL, ty, 'big_tiles\\')
# Generate tiles until enough are made
train_start = time.time()
if len(glob.glob(os.path.join(obg, 'HE*.jpg'))) >= ntrain:
    print('  Already done.')
else:
    while len(glob.glob(os.path.join(obg, 'HE*.jpg'))) < ntrain:
        numann, percann = combine_annotations_into_tiles(numann0, numann, percann, ctlist0, nblack, pthDL, ty, sxy)
        elapsed_time = time.time() - train_start
        print(f'  {len(glob.glob(os.path.join(obg, "HE*.jpg")))} of {ntrain} training images completed in {int(elapsed_time / 60)} minutes')

        baseclass1 = np.sum(percann0[:, :, 0])
        usedclass1 = np.sum(percann[:, :, 0])
        baseclass2 = np.sum(percann0[:, :, 1] == 1)
        usedclass2 = np.sum(percann[:, :, 1] == 2)

        tmp1 = usedclass1 / baseclass1 * 100
        tmp2 = usedclass2 / baseclass2 * 100

        for b, class_name in enumerate(classNames):
            print(f'  Used {tmp1[b]:.1f}% counts and {tmp2[b]:.1f}% unique annotations of {class_name}')
            
total_time_train_bigtiles = time.time() - train_start
hours, rem = divmod(total_time_train_bigtiles, 3600)
minutes, seconds = divmod(rem, 60)
print(f'  Elapsed time to create training big tiles: {hours}h {minutes}m {seconds}s')

print('')


Building training tiles...
Starting time for the while loop
Iteration: 1
 Bigtile occupancy rate: 0.00e+00 %
 type_: 0
Iteration: 2
 Bigtile occupancy rate: 0.00e+00 %
 type_: 2
Iteration: 3
 Bigtile occupancy rate: 1.57e-02 %
 type_: 3
Iteration: 4
 Bigtile occupancy rate: 1.57e-02 %
 type_: 7
Iteration: 5
 Bigtile occupancy rate: 2.55e-02 %
 type_: 8
Iteration: 6
 Bigtile occupancy rate: 2.55e-02 %
 type_: 9
Iteration: 7
 Bigtile occupancy rate: 4.20e-02 %
 type_: 8
Iteration: 8
 Bigtile occupancy rate: 4.20e-02 %
 type_: 2
Iteration: 9
 Bigtile occupancy rate: 4.83e-02 %
 type_: 1
Iteration: 10
 Bigtile occupancy rate: 4.83e-02 %
 type_: 2
Iteration: 11
 Bigtile occupancy rate: 7.22e-02 %
 type_: 0
Iteration: 12
 Bigtile occupancy rate: 7.22e-02 %
 type_: 10
Iteration: 13
 Bigtile occupancy rate: 9.41e-02 %
 type_: 9
Iteration: 14
 Bigtile occupancy rate: 9.41e-02 %
 type_: 1
Iteration: 15
 Bigtile occupancy rate: 1.14e-01 %
 type_: 3
Iteration: 16
 Bigtile occupancy rate: 1.14e-01

IndexError: boolean index did not match indexed array along dimension 1; dimension is 111 but corresponding boolean dimension is 112

In [ ]:
# Build validation tiles

ty = 'validation\\'
obg = os.path.join(pthDL, ty, 'big_tiles\\')
numann = numann0.copy()
percann = (numann0 > 0).astype(float)
percann = np.dstack((percann, percann))
percann0 = percann.copy()
validation_start_time = time.time()

print('')
print('Building validation tiles...')
if len(glob.glob(os.path.join(obg, 'HE*.jpg'))) >= nvalidate:
    print('Already done.')
else:
    while len(glob.glob(os.path.join(obg, 'HE*.jpg'))) < nvalidate:
        numann, percann = combine_annotations_into_tiles(numann0, numann, percann, ctlist0, nblack, pthDL, ty, sxy)
        elapsed_time = time.time() - validation_start_time
        print(f'{len(glob.glob(os.path.join(obg, "HE*.jpg")))} of {nvalidate} validation images completed in {int(elapsed_time / 60)} minutes')

        baseclass1 = np.sum(percann0[:, :, 0])
        usedclass1 = np.sum(percann[:, :, 0])
        baseclass2 = np.sum(percann0[:, :, 1] == 1)
        usedclass2 = np.sum(percann[:, :, 1] == 2)

        tmp1 = usedclass1 / baseclass1 * 100
        tmp2 = usedclass2 / baseclass2 * 100

        for b, class_name in enumerate(classNames):
            print(f'Used {tmp1[b]:.1f}% counts and {tmp2[b]:.1f}% unique annotations of {class_name}')
            
total_time_validation_bigtiles = time.time() - validation_start_time
hours, rem = divmod(total_time_validation_bigtiles, 3600)
minutes, seconds = divmod(rem, 60)
print(f'Elapsed time to create validation big tiles: {hours}h {minutes}m {seconds}s')
